In [11]:
import torch
import json
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from transformers import AutoModel, AutoTokenizer


In [41]:
from huggingface_hub import notebook_login
#huggingface upload
notebook_login()

Login successful
Your token has been saved to C:\Users\DESKTOP/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [13]:
modelName="klue/bert-base"
tokenizerName="klue/bert-base"
embeddingModelName=models.Transformer(modelName)
tokenizer = AutoTokenizer.from_pretrained(tokenizerName)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def readTrainData(path):
    trainDataset=[]
    with open(path,'rb')as file:
        STSdata=json.load(file)
    
    for item in STSdata:
        score=float(item['labels']['label'])/5.0

        inputEx=InputExample(
        texts=[item["sentence1"],item["sentence2"]],
        label=score
        )
        trainDataset.append(inputEx)
    
    return trainDataset

In [15]:
BATCH_SIZE=32
EPOCH=4
outputPath=(modelName+"_epoch-"+str(EPOCH)).replace("/","-")
outputPath='./outputs/'+outputPath

In [16]:
#pooling, mean 사용예정
pooler = models.Pooling(
    embeddingModelName.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [17]:
model = SentenceTransformer(modules=[embeddingModelName, pooler])

In [18]:
dataName="./dataset/klue-sts-v1.1_train.json"
trainDataset=readTrainData(dataName)

In [19]:
trainDataLoader=DataLoader(
trainDataset,
shuffle=True,
batch_size=BATCH_SIZE
)
trainLoss=losses.CosineSimilarityLoss(model=model)

In [126]:
evaluator=EmbeddingSimilarityEvaluator.from_input_examples(
trainDataset,
name="sts-vali"
)

In [ ]:
model.fit(
train_objectives=[(trainDataLoader, trainLoss)],
    evaluator=evaluator,
    epochs=EPOCH,
    evaluation_steps=1000,
    output_path=outputPath
 )

C:\Users\DESKTOP\anaconda3\envs\a2\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

In [20]:
tokenizer.save_pretrained("./model")
model.save("./model")

In [51]:
from huggingface_hub import HfApi
from huggingface_hub import upload_file, delete_file,upload_folder
model = AutoModel.from_pretrained("KDHyun08/TAACO_STS/STS.1.0v")

OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like KDHyun08/TAACO_STS/STS.1.0v is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [43]:
upload_folder(
    folder_path ="model",
    path_in_repo="STS.1.0v",
    repo_id="KDHyun08/TAACO_STS",
)

'https://huggingface.co/KDHyun08/TAACO_STS/tree/main/STS.1.0v'

In [ ]:
docs = ['어제는 아내의 생일이었다', '생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터 음식을 준비하였다. 주된 메뉴는 스테이크와 낙지볶음, 미역국, 잡채, 소야 등이었다', '스테이크는 자주 하는 음식이어서 자신이 준비하려고 했다', '앞뒤도 1분씩 3번 뒤집고 래스팅을 잘 하면 육즙이 가득한 스테이크가 준비되다', '아내도 그런 스테이크를 좋아한다. 그런데 상상도 못한 일이 벌이지고 말았다', '보통 시즈닝이 되지 않은 원육을 사서 스테이크를 했는데, 이번에는 시즈닝이 된 부챗살을 구입해서 했다', '그런데 케이스 안에 방부제가 들어있는 것을 인지하지 못하고 방부제와 동시에 프라이팬에 올려놓을 것이다', '그것도 인지 못한 체... 앞면을 센 불에 1분을 굽고 뒤집는 순간 방부제가 함께 구어진 것을 알았다', '아내의 생일이라 맛있게 구워보고 싶었는데 어처구니없는 상황이 발생한 것이다', '방부제가 센 불에 녹아서 그런지 물처럼 흘러내렸다', ' 고민을 했다. 방부제가 묻은 부문만 제거하고 다시 구울까 했는데 방부제에 절대 먹지 말라는 문구가 있어서 아깝지만 버리는 방향을 했다', '너무나 안타까웠다', '아침 일찍 아내가 좋아하는 스테이크를 준비하고 그것을 맛있게 먹는 아내의 모습을 보고 싶었는데 전혀 생각지도 못한 상황이 발생해서... 하지만 정신을 추스르고 바로 다른 메뉴로 변경했다', '소야, 소시지 야채볶음..', '아내가 좋아하는지 모르겠지만 냉장고 안에 있는 후랑크소세지를 보니 바로 소야를 해야겠다는 생각이 들었다. 음식은 성공적으로 완성이 되었다', '40번째를 맞이하는 아내의 생일은 성공적으로 준비가 되었다', '맛있게 먹어 준 아내에게도 감사했다', '매년 아내의 생일에 맞이하면 아침마다 생일을 차려야겠다. 오늘도 즐거운 하루가 되었으면 좋겠다', '생일이니까~']
document_embeddings = model.encode(docs)

query = '생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터 음식을 준비하였다'
query_embedding = model.encode(query)

top_k = min(10, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")